In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yale-face-database/subject05.surprised
/kaggle/input/yale-face-database/subject09.normal
/kaggle/input/yale-face-database/subject04.rightlight
/kaggle/input/yale-face-database/subject03.leftlight
/kaggle/input/yale-face-database/subject07.glasses
/kaggle/input/yale-face-database/subject03.normal
/kaggle/input/yale-face-database/subject11.rightlight
/kaggle/input/yale-face-database/subject02.surprised
/kaggle/input/yale-face-database/subject14.surprised
/kaggle/input/yale-face-database/subject12.centerlight
/kaggle/input/yale-face-database/subject01.sad
/kaggle/input/yale-face-database/subject09.glasses
/kaggle/input/yale-face-database/subject01.rightlight
/kaggle/input/yale-face-database/subject03.noglasses
/kaggle/input/yale-face-database/subject06.noglasses
/kaggle/input/yale-face-database/subject09.happy
/kaggle/input/yale-face-database/subject03.centerlight
/kaggle/input/yale-face-database/subject12.surprised
/kaggle/input/yale-face-database/subject15.surprised
/kaggl

## Imports Libraries:

In [6]:
import matplotlib.pyplot as plt
import os

## Retreiving Files

In [36]:
dir = "/kaggle/input/yale-face-database/data"

In [37]:
def list_files(dir, contains):
    return list(f for f in os.listdir(dir) if contains in f)

In [42]:
files = list_files(dir, 'subject01')
print(files)

['subject01.sad', 'subject01.rightlight', 'subject01.glasses', 'subject01.centerlight', 'subject01.normal', 'subject01.noglasses', 'subject01.happy', 'subject01.surprised', 'subject01.leftlight', 'subject01.wink', 'subject01.sleepy']


The task is to develop a system that can accurately identify the subject in any given image. For example, if an image of subject14 is fed under any condition (like subject14.surprised), the system should correctly identify it as subject14.

## Converting to .png